In [1]:
# Imports
import numpy as np
import pandas as pd
import time
import datetime
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# Defines
YEAR = 2017
THRESHOLD = .60
FILL_NA = 0
DATE_CONVERSION = 'timestamps'
PREPROCESSING = 'MinMax'
KFOLD_SPLITS = 10

In [3]:
# Functions
def check_na(train):
    # Finds the number of missing values in each column
    num_of_na = [train.loc[:,col].isnull().sum() for col in train]
    # Divide by rows for proportion 
    prop_na = [num / train.shape[0] for num in num_of_na]
    # Put the proporitons and column names into a df and sort
    na_df = pd.DataFrame({'prop_na' : prop_na, 'column' : train.columns}).sort_values('prop_na')
    return na_df

In [4]:
# Read csvs
if YEAR == 2016:
    properties = pd.read_csv('properties_2016.csv', low_memory = False)
    train = pd.read_csv('train_2016_v2.csv', low_memory = False)
elif YEAR == 2017:
    properties = pd.read_csv('properties_2017.csv', low_memory = False)
    train = pd.read_csv('train_2017.csv', low_memory = False)
# train has Y and properties has features
# Find row intersection of train and properties
train = train.merge(properties, on = 'parcelid', how = 'left')

In [5]:
print('BEFORE')
print(check_na(train))
# Remove all columns above the THRESHOLD
train = train.loc[:, (train.isnull().sum(axis=0) <= (train.shape[0]*THRESHOLD))]
print('AFTER')
print(check_na(train))

BEFORE
     prop_na                        column
0   0.000000                      parcelid
1   0.000000                      logerror
2   0.000000               transactiondate
27  0.000438                     longitude
34  0.000438     propertycountylandusecode
35  0.000438         propertylandusetypeid
37  0.000438        rawcensustractandblock
39  0.000438                regionidcounty
42  0.000438                       roomcnt
19  0.000438                          fips
7   0.000438                    bedroomcnt
6   0.000438                   bathroomcnt
54  0.000438                assessmentyear
26  0.000438                      latitude
53  0.000451             taxvaluedollarcnt
55  0.000464         landtaxvaluedollarcnt
56  0.000502                     taxamount
41  0.001082                   regionidzip
52  0.001920    structuretaxvaluedollarcnt
13  0.003028  calculatedfinishedsquarefeet
59  0.003621           censustractandblock
49  0.003917                     yearbuilt
21  

In [6]:

# Convert transactiondate strings into floats
date_strings = (train.values[:,2])
date_converted = []
if DATE_CONVERSION == 'timestamps':
    for string in date_strings:
        date_converted.append(time.mktime(datetime.datetime.strptime(string, "%Y-%m-%d").timetuple()))
train['transactiondate'] = np.asarray(date_converted)
# Drop the columns with string and int
train = train.drop(columns=['propertycountylandusecode', 'propertyzoningdesc'])

In [7]:
# Preprocessing

# scales a column x of a pandas dataframe
def minmaxscaler_dropna(x):
    # Formula from:
    # https://stackoverflow.com/questions/39758449/normalise-between-0-and-1-ignoring-nan
    return((x - x.min()) / (x.max() - x.min()))

# TODO: use another scale function, like normalize, that has mean 0

def normalize_scaler(x):
    return((x - np.mean(x)) / np.std(x))


if PREPROCESSING == 'MinMax':
#     scaler = MinMaxScaler()
#     scaler.fit(x)
#     x = scaler.transform(x)
    train2 = [normalize_scaler(train[col]) for col in train.columns]



In [8]:
type(train2)

list

In [9]:
train2 = pd.DataFrame(train2).transpose()
train2.head()

,parcelid,logerror,transactiondate,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
0,0.366531,0.051730,-1.926531,1.205452,0.830162,NaN,1.208154,1.378122,1.433853,0.490952,...,0.103151,-0.522822,NaN,1.235173,1.286546,0.815443,NaN,0.479819,0.657738,0.061276
1,1.149597,0.227427,-1.926531,-1.302762,-0.923497,NaN,-1.343692,-0.335276,-0.316033,2.997325,...,0.135048,1.247925,NaN,-0.067730,-0.439570,-0.039998,NaN,0.151908,-0.042403,0.400003
2,0.334907,-0.066548,-1.926531,-0.299476,-0.046668,NaN,-0.322954,-0.567921,-0.553633,0.490952,...,0.129512,1.602074,NaN,-0.277876,-0.451337,0.114145,NaN,0.361944,0.064537,0.061007
3,-0.235855,-0.703185,-1.926531,0.703809,0.830162,0.851442,0.697785,0.619406,0.658980,-0.569436,...,-0.067666,-0.522822,-0.094285,0.058357,-0.348784,-0.527701,NaN,-0.537680,-0.552963,-0.080656
4,-0.602665,-0.057433,-1.926531,0.703809,-0.046668,0.851442,0.697785,-0.495613,-0.479784,-0.569436,...,-0.035770,-0.522822,-0.094285,-0.193818,-0.501716,-0.567065,NaN,-0.518397,-0.584897,-0.081808


In [10]:
y = train2.values[:,1]
y = y.reshape(y.shape[0],1)
x = train2.values[:,2:]

In [31]:
# x

In [32]:
# # scales a column x of a pandas dataframe
# def minmaxscaler_dropna(x):
#     # Formula from:
#     # https://stackoverflow.com/questions/39758449/normalise-between-0-and-1-ignoring-nan
#     (x - x.min()) / (x.max() - x.min())


# if PREPROCESSING == 'MinMax':
# #     scaler = MinMaxScaler()
# #     scaler.fit(x)
# #     x = scaler.transform(x)
#     x2 = [minmaxscaler_dropna(x.col) for col in x.columns]
# # x.col =     
# x2

In [11]:
# Replace all NAs with number defined in FILL_NA
train = train2.fillna(FILL_NA)

In [12]:
# KFolds
train_index_array = []
test_index_array = []
kf = KFold(n_splits = KFOLD_SPLITS, shuffle = True, random_state = 1)
for train_index, test_index in kf.split(x):
    print("TRAIN:", train_index, "TEST:", test_index)
    train_index_array.append(train_index)
    test_index_array.append(test_index)
MY_INDEX = 6
x_train, x_test = x[train_index_array[MY_INDEX]], x[test_index_array[MY_INDEX]]
y_train, y_test = y[train_index_array[MY_INDEX]], y[test_index_array[MY_INDEX]]

TRAIN: [    0     2     3 ... 77609 77611 77612] TEST: [    1    28    30 ... 77602 77607 77610]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [   10    24    25 ... 77586 77593 77595]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    3    12    13 ... 77582 77603 77608]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    8    23    40 ... 77567 77597 77599]
TRAIN: [    0     1     3 ... 77610 77611 77612] TEST: [    2    11    22 ... 77569 77574 77605]
TRAIN: [    0     1     2 ... 77609 77610 77611] TEST: [   33    37    39 ... 77585 77601 77612]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [   26    35    36 ... 77587 77588 77591]
TRAIN: [    1     2     3 ... 77609 77610 77612] TEST: [    0     4     6 ... 77600 77606 77611]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    5    41    49 ... 77592 77604 77609]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    9    15    18 ... 77589 77594 77596]


In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(9, input_dim = 27, activation = 'relu'),
        tf.keras.layers.Dense(1, activation = 'linear')
    ])
sgd = tf.keras.optimizers.SGD(lr=0.1)
model.compile(loss = 'mse', optimizer = sgd)
model.fit(x_train, y_train, epochs = 500, batch_size = 32, verbose = 1)

Epoch 1/500
69852/69852 [==============================] - 2s 35us/step - loss: nan
Epoch 2/500
69852/69852 [==============================] - 2s 31us/step - loss: nan
Epoch 3/500
69852/69852 [==============================] - 2s 30us/step - loss: nan
Epoch 4/500
69852/69852 [==============================] - 2s 36us/step - loss: nan
Epoch 5/500
69852/69852 [==============================] - 2s 35us/step - loss: nan
Epoch 6/500
69852/69852 [==============================] - 3s 46us/step - loss: nan
Epoch 7/500
69852/69852 [==============================] - 3s 39us/step - loss: nan
Epoch 8/500
69852/69852 [==============================] - 3s 44us/step - loss: nan


In [ ]:
model.evaluate(x_test, y_test, verbose = 1)